# Climate Change Risk Dashboard 
#### Realtime, processing in external module

This dashboard is an example of highlighting risks due to climate change. In this case we study the mainland US and explore risk as a function of global heating, over the period of 1860 - 2100, and comparing the climatic future scenarios A1B and E1.

This version of the dashboard calculates statistics (specifically in-state average temperature) in realtime, rather than being fully reliant on pre-computed data to populate the dashboard.

In [ ]:
import cartopy.crs as ccrs

from distributed import Client, LocalCluster

import colorcet as cc
import geoviews.feature as gf
import hvplot.pandas
import panel as pn
import param

from processing import DataHolder

In [ ]:
pn.extension()

## Load all data

In [ ]:
processor = DataHolder()
state_codes = processor.state_codes

## Interactive and Realtime Plot

Produce an interactive plot that calculates mean temperatures within states for a selected year and RCP scenario in an on-demand fashion.

### Full interface

This takes the `ClimateRiskInterface` from the non-realtime dashboard, strips out extra calculations such as risk calculation, and adds a simple method that calculates mean temperature within a state for a given year and RCP in realtime. We wrap the plot in a `pn.panel` call so that we can put a spinning-wheel loading indicator over the plot while the calculations run in realtime, and so that the interface does not lock up while the calculations execute.

Note that by adding calculated values to the geodataframe `us_states`, we can also use this as a simple cache to avoid recalculating results that have already been calculated.

In [ ]:
# Create a local dask distributed cluster for processing statistics in realtime.

# Important: multiple threads cause distributed processing to lock up.
lc = LocalCluster(n_workers=8, threads_per_worker=1)
c = Client(lc)
c

In [ ]:
class ClimateRiskInterfaceRT(param.Parameterized):
    # Set params that add specific interactivity to the plot.
    rcp = param.ObjectSelector(default="a1b", objects=["a1b", "e1"], label="RCP")
    year = param.Integer(default=2021, bounds=(1860, 2099))
    
    # Set some defaults.
    tmin, tmax = 255, 308

    def prepare_data(self, col_ref):
        processor.calculate_one_column(col_ref)
        return processor.us_states[col_ref]
    
    @param.depends("rcp", "year")
    def plot(self):
        """Plot all the elements of the map."""
        col_ref = f"{self.rcp},{self.year}"
        polyplot = processor.us_states.hvplot.polygons(geo=True,
                                                       c=self.prepare_data(col_ref),
                                                       hover_cols=["name"]).opts(
            toolbar="above", clim=(self.tmin, self.tmax),
            projection=ccrs.LambertConformal(),
            cmap=cc.cm.CET_L4, colorbar=True)
            
        result = (gf.coastline * polyplot)
        return result.opts(width=1200, height=800)

In [ ]:
interface = ClimateRiskInterfaceRT()
pn.Row(interface.param, pn.panel(interface.plot, loading_indicator=True)).servable()